In [1]:
import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

In [3]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [4]:
import warnings
warnings.filterwarnings("ignore")

### Data preprocessing

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
edges_df = pd.read_excel(r'~/Downloads/Traffic Speed Prediction Street View Images/Adjacent_Matrix.xls')
target = list(edges_df['TARGET_FID'])
join = list(edges_df['JOIN_FID'])

In [16]:
features_df = pd.read_csv(r'~/Downloads/Traffic Speed Prediction Street View Images/Features.csv')
feature_list = []
for i in range(features_df.shape[0]):
    feature = list(features_df.iloc[i][2: 367])
    feature_list.append(feature)
X_Image = torch.as_tensor(feature_list)

In [40]:
#Create traffic speed for each hour
speed_df = pd.read_csv(r'~/Downloads/Traffic Speed Prediction Street View Images/Y_2.csv')
speed_list = []
for i in range(speed_df.shape[0]):
    speed = list(speed_df.iloc[i][5:29])
    speed_list.append(speed)
Y = torch.as_tensor(speed_list)
X_Speed = Y[:, :12]
Y = Y[:, 12:]

In [42]:
Mask_df = pd.read_csv(r'~/Downloads/Traffic Speed Prediction Street View Images/Y_2.csv')
Train_Mask_list = list(Mask_df['Train_Mask'])
Train_Mask = torch.as_tensor(Train_Mask_list)

Val_Mask_list = list(Mask_df['Validation_Mask'])
Val_Mask = torch.as_tensor(Val_Mask_list)

Test_Mask_list = list(Mask_df['Test_Mask'])
Test_Mask = torch.as_tensor(Test_Mask_list)

### Prepare Dataset

In [5]:
from torch.utils.data import Dataset

In [7]:
class TrafficSpeedDataset(Dataset):
    def __init__(self, data_list, transform=None):
        self.data = data_list
        self.transform = transform
        self.x
        self.y
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        x
        y
        return x, y

In [9]:
training_dataloader = None
test_dataloader = None

### Network Construction

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, time_number, LSTM_hidden):
        super(Net, self).__init__()
        self.time_number = time_number
        self.LSTM_hidden = LSTM_hidden
        self.num_layers = 4
        
        #graph part
        self.conv1 = GCNConv()
        self.conv2 = GCNConv()
        self.conv3 = GCNConv()
        
        #Lstm part
        self.lstm = nn.LSTM(
            input_size = self.time_number,
            hidden_size = self.LSTM_hidden,
            num_layers = self.num_layers
        )
        
        
    
    def forward(self, data):
        pass

### Training process

In [ ]:
def train(train_dataloader, EPOCHS):
    for e in range(EPOCHS):
        pass
def test(test_dataloader, model, optimizer):
    pass


In [ ]:
device = torch.device('cuda')

model = Net()
model.to(device)

learning_rate = 0.001
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training
train()
test()